<a href="https://colab.research.google.com/github/ShadiFarzankia/Human-Value-Detection-with-NLP/blob/master/Human_Value_Detection_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human Value Detection


### Required Libraries


In [2]:
import json
import pandas as pd

### Loading the Data

In [10]:
#Loading the Values
with open('values.json') as value_file:
   values_data = json.load(value_file)

print(values_data)

{'values': [{'alternativeName': 'Creativity/Imagination', 'name': 'Be creative', 'id': 'SDT1', 'level2': 'Self-direction: thought', 'level3': ['Openness to change'], 'level4a': ['Personal focus'], 'level4b': ['Growth, Anxiety-free'], 'descriptions': ['allowing for more creativity or imagination', 'being more creative', 'fostering creativity', 'promoting imagination']}, {'alternativeName': 'Curious/Interested', 'name': 'Be curious', 'id': 'SDT2', 'level2': 'Self-direction: thought', 'level3': ['Openness to change'], 'level4a': ['Personal focus'], 'level4b': ['Growth, Anxiety-free'], 'descriptions': ['being the more interesting option', 'fostering curiosity', 'making people more keen to learn', 'promoting discoveries', 'sparking interest']}, {'alternativeName': 'Freedom of thought', 'name': 'Have freedom of thought', 'id': 'SDT3', 'level2': 'Self-direction: thought', 'level3': ['Openness to change'], 'level4a': ['Personal focus'], 'level4b': ['Growth, Anxiety-free'], 'descriptions': ['al

In [17]:
values_data['values'][0]

{'alternativeName': 'Creativity/Imagination',
 'name': 'Be creative',
 'id': 'SDT1',
 'level2': 'Self-direction: thought',
 'level3': ['Openness to change'],
 'level4a': ['Personal focus'],
 'level4b': ['Growth, Anxiety-free'],
 'descriptions': ['allowing for more creativity or imagination',
  'being more creative',
  'fostering creativity',
  'promoting imagination']}

In [18]:
values_data['values'][1]

{'alternativeName': 'Curious/Interested',
 'name': 'Be curious',
 'id': 'SDT2',
 'level2': 'Self-direction: thought',
 'level3': ['Openness to change'],
 'level4a': ['Personal focus'],
 'level4b': ['Growth, Anxiety-free'],
 'descriptions': ['being the more interesting option',
  'fostering curiosity',
  'making people more keen to learn',
  'promoting discoveries',
  'sparking interest']}

In [19]:
values_data['values'][2]

{'alternativeName': 'Freedom of thought',
 'name': 'Have freedom of thought',
 'id': 'SDT3',
 'level2': 'Self-direction: thought',
 'level3': ['Openness to change'],
 'level4a': ['Personal focus'],
 'level4b': ['Growth, Anxiety-free'],
 'descriptions': ['allowing people to figure things out on their own',
  'allowing people to make up their mind',
  'resulting in less censorship',
  "resulting in less influence on people's thoughts"]}

### Extrating Values

In [23]:
values = { "1":list(), "2":list(), "3":list(), "4a":list(), "4b":list() } 
for value in values_data["values"]:
  values["1"].append(value["name"])
  values["2"].append(value["level2"])
  for valueLevel3 in value["level3"]:
    values["3"].append(valueLevel3)
  for valueLevel4a in value["level4a"]:
      values["4a"].append(valueLevel4a)
  for valueLevel4b in value["level4b"]:
      values["4b"].append(valueLevel4b)
  values["1"] = sorted(values["1"])
  values["2"] = sorted(values["2"])
  values["3"] = sorted(values["3"])
  values["4a"] = sorted(values["4a"])
  values["4b"] = sorted(values["4b"])

In [24]:
values

{'1': ['Be ambitious',
  'Be behaving properly',
  'Be broadminded',
  'Be capable',
  'Be choosing own goals',
  'Be compliant',
  'Be courageous',
  'Be creative',
  'Be curious',
  'Be daring',
  'Be forgiving',
  'Be helpful',
  'Be holding religious faith',
  'Be honest',
  'Be honoring elders',
  'Be humble',
  'Be independent',
  'Be intellectual',
  'Be just',
  'Be logical',
  'Be loving',
  'Be neat and tidy',
  'Be polite',
  'Be protecting the environment',
  'Be respecting traditions',
  'Be responsible',
  'Be self-disciplined',
  'Have a comfortable life',
  'Have a good reputation',
  'Have a safe country',
  'Have a sense of belonging',
  'Have a stable society',
  'Have a varied life',
  'Have a world at peace',
  'Have a world of beauty',
  'Have an exciting life',
  'Have an objective view',
  'Have equality',
  'Have freedom of action',
  'Have freedom of thought',
  'Have good health',
  'Have harmony with nature',
  'Have influence',
  'Have life accepted as is',

In [25]:
#Loading the Arguments
dataframe = pd.read_csv('arguments.tsv', encoding='utf-8', sep='\t', header=0)

print(dataframe)

     Argument ID   Part  Usage                      Conclusion       Stance  \
0         A01001    usa  train  Entrapment should be legalized  in favor of   
1         A01002    usa  train     We should ban human cloning  in favor of   
2         A01003    usa  train      We should abandon marriage      against   
3         A01004    usa  train       We should ban naturopathy      against   
4         A01005    usa  train         We should ban fast food  in favor of   
...          ...    ...    ...                             ...          ...   
5265      D27096  india   test    Nepotism exists in Bollywood      against   
5266      D27097  india   test    Nepotism exists in Bollywood  in favor of   
5267      D27098  india   test         India is safe for women  in favor of   
5268      D27099  india   test         India is safe for women  in favor of   
5269      D27100  india   test         India is safe for women      against   

                                                Pre

In [30]:
dataframe[:][:]

,Argument ID,Part,Usage,Conclusion,Stance,Premise
0,A01001,usa,train,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...
1,A01002,usa,train,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...
2,A01003,usa,train,We should abandon marriage,against,marriage is the ultimate commitment to someone...
3,A01004,usa,train,We should ban naturopathy,against,it provides a useful income for some people
4,A01005,usa,train,We should ban fast food,in favor of,fast food should be banned because it is reall...
...,...,...,...,...,...,...
5265,D27096,india,test,Nepotism exists in Bollywood,against,Star kids also have an upbringing which is sur...
5266,D27097,india,test,Nepotism exists in Bollywood,in favor of,Movie stars of Bollywood often launch their ch...
5267,D27098,india,test,India is safe for women,in favor of,Evil historic practices on women in the pre an...
5268,D27099,india,test,India is safe for women,in favor of,Women of our country have been and are achievi...


In [36]:
df_model_head = pd.DataFrame(dataframe, columns=['Usage'])

In [32]:
df_model_head

,Usage
0,train
1,train
2,train
3,train
4,train
...,...
5265,test
5266,test
5267,test
5268,test
